# Revista de Gestão e Contabilidade da UFPI

## Bibliotecas importadas

In [19]:
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/labfis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
plt.style.use('seaborn')

## Funções básicas

In [21]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [22]:
def listar_links_pagina(query, url_search_text):
    url_search = url_search_text.replace('__query__', query)
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    tbl_results = soup.find('table', {'class': 'listing'})
    a_links = []
    for a in tbl_results.find_all('a'):
           if 'Resumo' in a.text:
                a_links.append(a['href'])
    return a_links



In [23]:
url_search_text = 'https://revistas.ufpi.br/index.php/gecont/search/search?query=__query__&authors=&title=&abstract=&galleyFullText=&suppFiles=&dateFromMonth=&dateFromDay=&dateFromYear=2014&dateToMonth=&dateToDay=&dateToYear=2021&dateToHour=23&dateToMinute=59&dateToSecond=59&discipline=&subject=&type=&coverage=&indexTerms='

links = listar_links_pagina('contabilidade', url_search_text)

links[0]

'https://revistas.ufpi.br/index.php/gecont/article/view/6292'

In [24]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [25]:
def captura_titulo(soup):
    try:
        title_tag = soup.find('div', {'id': 'articleTitle'})
        title = title_tag.find('h3').text
    except:
        title = ''
    return {'Título': title}

In [26]:
def captura_ano(soup):
    try:
        year_tags = soup.find('div', {'id': 'breadcrumb'})
        year_tag = year_tags.find_all('a')
        year_text = year_tag[1].text
        year = year_text[-5:-1]
    except:
        year = ''
    return {'Data de Publicação': year}

In [27]:
def captura_resumo(soup):
    try:
        abstract_tag = soup.find('div', {'id': 'articleAbstract'})
        abstract = abstract_tag.find('div').text
    except:
        abstract = ''
    return {'Resumo': abstract}

In [28]:
def captura_palavras_chave(soup):
    try:
        kw_tag = soup.find('div', {'id': 'articleSubject'})
        kw = kw_tag.find('div').text.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [29]:
def captura_autores(soup):
    autores = []
    try:
        authors_tag = soup.find_all('div', {'class': 'authorBio'})
        
        for p in authors_tag:
            try:
                autor = p.find('em').text
            except:
                autor = ''
            autores.append(autor)
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [30]:
def captura_afiliacao(soup):
    affiliation_list = []
    try:
        authors_tag = soup.find_all('div', {'class': 'authorBio'})
        for p in authors_tag:
            try:
                affiliation = p.find_all('br')[-2].next_sibling
                affiliation = affiliation.replace('\t', '')
            except:
                affiliation = ''
            affiliation_list.append(affiliation)
    except:
        pass
    affiliation_list = completar_lista(affiliation_list)
    key_affiliation = [f"Afilição {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, affiliation_list))

In [50]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'periódico': 'Revista de Gestão e Contabilidade da UFPI'}
    url_search_text = 'https://revistas.ufpi.br/index.php/gecont/search/search?query=__query__&authors=&title=&abstract=&galleyFullText=&suppFiles=&dateFromMonth=&dateFromDay=&dateFromYear=2014&dateToMonth=&dateToDay=&dateToYear=2021&dateToHour=23&dateToMinute=59&dateToSecond=59&discipline=&subject=&type=&coverage=&indexTerms='

    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links_pagina(query, url_search_text)
        
    resultados = []
    for link in a_links:
        link_dict = {'url': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_ano(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y')
    return df

## Pesquisa por artigos

In [ ]:
key_words_list = ['contabilidade ambiental', 'balanço social']
df = realizar_consulta(key_words_list)
df.head()

In [ ]:
df.to_csv('../data/GECONT.csv', index=False)

In [ ]:
df